In [2]:


%pip install --upgrade --quiet  docx2txt langchain-community

Note: you may need to restart the kernel to use updated packages.


In [31]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
documentList = loader.load_and_split(text_splitter=text_splitter)

In [26]:

from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
load_dotenv(dotenv_path='.venv/pyvenv.cfg')
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [27]:
from langchain_chroma import Chroma

database = Chroma.from_documents(persist_directory="./chroma", collection_name='chroma-tax', documents=documentList , embedding=embedding)

NameError: name 'documentList' is not defined

In [38]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'


retrived_docs = database.similarity_search(query)

retrived_docs

[Document(id='284bd903-bb85-4f28-8194-ee60f2a2bd6d', metadata={'source': './tax.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금

In [39]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo')

In [40]:
prompt = f"""[Identity]

- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]
{retrived_docs}

Question: {query}
"""

In [41]:
ai_message = llm.invoke(prompt)


ai_message.content

'연봉 5천만원인 직장인의 소득세는 대통령령으로 정해진 세율에 따라 계산됩니다. 세율은 소득에 따라 다르며, 일반적으로 소득이 증가할수록 세율도 증가합니다. 따라서 정확한 소득세 계산을 위해서는 해당 국가의 세법을 확인하고 해당 연봉에 대한 세율을 적용하여 계산해야 합니다. 세부적인 세금 계산을 위해서는 해당 국가의 세법을 상세히 살펴보는 것이 필요합니다.'

In [29]:

#============= langchain hub를 활용 RetrievalQA ====================
from langchain_chroma import Chroma

database = Chroma(collection_name='chroma-tax' , persist_directory="./chroma" , embedding_function=embedding)

In [30]:
from langchain_openai import ChatOpenAI
query = "연봉 5천만원의 직장인의 소득세는 얼마인가요?"
llm = ChatOpenAI(model='gpt-3.5-turbo')

In [38]:

from dotenv import load_dotenv
from langchain import hub
import os
load_dotenv(dotenv_path='.venv/pyvenv.cfg')
api_key = os.environ['OPENAI_API_KEY']
prompt = hub.pull("rlm/rag-prompt",api_key=api_key)

In [42]:
# 어느 백터 디비를 사용해도 langchain을 사용한다면 같은 코드로 수행 가능

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs={"prompt" : prompt}
)

In [44]:
ai_message = qa_chain({"query" : query})

In [45]:
ai_message

{'query': '연봉 5천만원의 직장인의 소득세는 얼마인가요?',
 'result': '제공된 문맥에서 연봉 5천만원의 직장인의 소득세는 알 수 없습니다.'}